In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [84]:
df = pd.read_csv("Web-Layered-Navigation-By-Category.csv")
df.columns

Index(['Link', 'Taxonomy', 'H1 Title', 'WebstaurantPlus', 'Quick Ship',
       'Brand', 'Type', 'Chemical Form', 'Parts', 'Size', 'Material',
       'Thickness', 'Capacity', 'Height', 'Features'],
      dtype='object')

In [3]:
df2 = pd.read_csv("data_without_sidebar.csv")
df2.columns

Index(['Link', 'Taxonomy', 'H1 Title'], dtype='object')

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

urls = df2['Link']

data_columns = ["Link", "Taxonomy", "H1 Title"]
data = pd.DataFrame(columns=data_columns)
taxonomies = []
titles = []

with open("error_urls.txt", "w") as error_file:
    counter = 0
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            print(counter, end=' ')
            counter += 1

            soup = BeautifulSoup(response.text, 'html.parser')

            breadcrumb_element = soup.find_all('ol', class_='breadcrumb')
            h1_heading = soup.find('h1', class_='page-header')

            if breadcrumb_element:
                breadcrumb = ' > '.join([item.text.strip() for item in breadcrumb_element[0].find_all('span')])
                breadcrumb = fix_breadcrumb(breadcrumb)
                taxonomies.append(breadcrumb)
                print(breadcrumb)
            else:
                taxonomies.append("")

            if h1_heading:
                title = h1_heading.text.strip()
                titles.append(title)
                print(title)
            else:
                titles.append("")

        except requests.exceptions.RequestException as e:
            print(f"Error while requesting {url}: {e}")
            error_file.write(f"Error while requesting {url}: {e}\n")
        except Exception as e:
            print(f"Error while processing {url}: {e}")
            error_file.write(f"Error while processing {url}: {e}\n")

data.to_csv('new_data.csv', index=False)

0 WebstaurantStore > Smallwares > Kitchen Supplies > Food Safety Supplies 
Food Safety Supplies
1 WebstaurantStore > Restaurant Equipment > Cooking Equipment > Commercial Restaurant Ranges 
2 WebstaurantStore > Janitorial Supplies > Cleaning Tools & Supplies > Bottles, Sprayers, Hoses, and Buckets 
Bottles, Sprayers, Hoses, and Buckets
3 WebstaurantStore > Food & Beverage > Specialty Foods > Gluten Free Foods 


KeyboardInterrupt: 

In [8]:
data['Taxonomy'] = taxonomies
data['H1 Title'] = titles
data

,Link,Taxonomy,H1 Title,H1 Titles
0,NaN,WebstaurantStore > Smallwares > Kitchen Suppli...,WebstaurantStore > Smallwares > Kitchen Suppli...,WebstaurantStore > Smallwares > Kitchen Suppli...
1,NaN,Food Safety Supplies,Food Safety Supplies,Food Safety Supplies
2,NaN,WebstaurantStore > Restaurant Equipment > Cook...,WebstaurantStore > Restaurant Equipment > Cook...,WebstaurantStore > Restaurant Equipment > Cook...
3,NaN,,,
4,NaN,WebstaurantStore > Janitorial Supplies > Clean...,WebstaurantStore > Janitorial Supplies > Clean...,WebstaurantStore > Janitorial Supplies > Clean...
...,...,...,...,...
969,NaN,Bakery Boxes and Containers,Bakery Boxes and Containers,Bakery Boxes and Containers
970,NaN,WebstaurantStore > Storage / Transport > Janit...,WebstaurantStore > Storage / Transport > Janit...,WebstaurantStore > Storage / Transport > Janit...
971,NaN,Janitorial Carts & Transport Equipment,Janitorial Carts & Transport Equipment,Janitorial Carts & Transport Equipment
972,NaN,WebstaurantStore > Disposables > Consumer Pack...,WebstaurantStore > Disposables > Consumer Pack...,WebstaurantStore > Disposables > Consumer Pack...


# Full Scraper

In [5]:
def fix_breadcrumb(breadcrumb_text):
    breadcrumb_parts = [part.strip() for part in breadcrumb_text.split(" > Current:")]

    unique_parts = []
    for part in breadcrumb_parts:
        if part not in unique_parts:
            unique_parts.append(part)

    formatted_breadcrumb = " > ".join(unique_parts)
    
    formatted_breadcrumb = formatted_breadcrumb[:-2]

    return formatted_breadcrumb

In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

urls = unique_links

data_columns = ["Link", "Taxonomy", "H1 Title"]
data = pd.DataFrame(columns=data_columns)

with open("error_urls.txt", "w") as error_file:
    counter = 0
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            print(counter, end=' ')
            counter += 1

            soup = BeautifulSoup(response.text, 'html.parser')

            breadcrumb_element = soup.find_all('ol', class_='breadcrumb')
            h1_heading = soup.find('h1', class_='page-header')
            sidebar = soup.find(id="sidebar")

            if breadcrumb_element:
                breadcrumb = ' > '.join([item.text.strip() for item in breadcrumb_element[0].find_all('span')])
                breadcrumb = fix_breadcrumb(breadcrumb)

            if h1_heading:
                title = h1_heading.text.strip()

            if sidebar:
                filter_items = sidebar.find_all("li")

                filter_dict = {"Link": url, "Taxonomy": breadcrumb, "H1 Title": title}

                filter_names = []
                for filter_item in filter_items:
                    filter_name = filter_item.find("span").text.strip()
                    print(filter_name)

                    option_items = filter_item.find_all("li")
                    options = [option_item.find("span").text.strip() for option_item in option_items]
                    options_string = "; ".join(options)
                    print(options_string)

                    if options:
                        filter_names.append(filter_name)
                        filter_dict[filter_name] = options_string

                data_columns = list(set(data_columns + filter_names))

                for column in data_columns:
                    if column not in filter_dict:
                        filter_dict[column] = ""

                data = data.append(filter_dict, ignore_index=True)

        except requests.exceptions.RequestException as e:
            print(f"Error while requesting {url}: {e}")
            error_file.write(f"Error while requesting {url}: {e}\n")
        except Exception as e:
            print(f"Error while processing {url}: {e}")
            error_file.write(f"Error while processing {url}: {e}\n")

data.to_csv('data3.csv', index=False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 

KeyboardInterrupt: 

In [90]:
import pandas as pd

df1 = pd.read_csv("final_data.csv", dtype=str)
df2 = pd.read_csv("data3.csv", dtype=str)

combined_df = pd.concat([df1, df2], axis=0)

combined_df.to_csv("final_data2.csv", index=False)

In [86]:
links1 = set(df['Link'])
links2 = set(combined_df['Link'])

links_only_in_df1 = links1 - links2

links_only_in_df2 = links2 - links1

unique_links = list(links_only_in_df1 | links_only_in_df2)

In [88]:
len(unique_links)

487

In [92]:
file_name = "leftover_urls.txt"

with open(file_name, "w") as file:
    for item in unique_links:
        file.write(f"{item}\n")